In [1]:
from rdflib import Graph, SKOS, Literal, URIRef, RDF
import utils
import warnings

In [5]:
RDF.type

rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type')

In [2]:
ontologies, label_uri_dict, uri_label_dict = utils.load_ontologies()
g_metadata = Graph().parse("../ontologies/scientific_metadata.json")

In [8]:

example_query = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX emmo: <http://emmo.info/emmo#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX echem: <http://emmo.info/electrochemistry#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dc: <http://purl.org/dc/elements/1.1/> 
PREFIX battery: <http://emmo.info/battery#> 

SELECT *
WHERE {
   ?subject rdf:type battery:U32e2d_Ecwec_ec4542.
}
"""

In [26]:
battery_cell = "battery_68ed592a_7924_45d0_a108_94d6275d57f0"
ncm = "http://emmo.info/emmo#EMMO_4950a41c_ae32_4ce9_9db9_0076bbcac18f"

query1 = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX emmo: <http://emmo.info/emmo#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX echem: <http://emmo.info/electrochemistry#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dc: <http://purl.org/dc/elements/1.1/> 
PREFIX battery: <http://emmo.info/battery#> 

SELECT ?instance
WHERE {
    ?instance rdfs:type emmo:EMMO_4950a41c_ae32_4ce9_9db9_0076bbcac18f.
}
"""

query2 = """ 
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX emmo: <http://emmo.info/emmo#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX echem: <http://emmo.info/electrochemistry#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dc: <http://purl.org/dc/elements/1.1/> 
PREFIX battery: <http://emmo.info/battery#> 

SELECT ?instance
WHERE {
    ?instance rdfs:subClassOf echem:electrochemistry_10359c79_cdf2_401a_83ee_d22e7fbb2ad1.
}
"""


Subclass queries in ontology work, but not istance queries on the json-ld file. Work on this

In [31]:
qres = ontologies.query(query2)
for result in qres:
    iri = str(result[0])
    print(uri_label_dict[iri])


Electrogravimetry
Impedimetry
Amperometry
Dielectrometry
Coulometry
Conductometry
Potentiometry


[]

In [3]:
dict_types = utils.get_all_ld_types()
dict_types 

{'BatteryCell': 'http://emmo.info/battery#battery_68ed592a_7924_45d0_a108_94d6275d57f0',
 'Mass': 'http://emmo.info/emmo#EMMO_ed4af7ae_63a2_497e_bb88_2309619ea405',
 'Real': 'http://emmo.info/emmo#EMMO_18d180e4_5e3e_42f7_820c_e08951223486',
 'Width': 'http://emmo.info/emmo#EMMO_e4de48b1_dabb_4490_ac2b_040f926c64f0',
 'Height': 'http://emmo.info/emmo#EMMO_08bcf1d6_e719_46c8_bb21_24bc9bf34dba',
 'Thickness': 'http://emmo.info/emmo#EMMO_43003c86_9d15_433b_9789_ee2940920656',
 'PositiveElectrode': 'http://emmo.info/electrochemistry#electrochemistry_aff732a9_238a_4734_977c_b2ba202af126',
 'NumberOfEntities': 'http://emmo.info/emmo#EMMO_41efdf5d_0c9c_4ea0_bb65_f8236e663be5',
 'CurrentCollector': 'http://emmo.info/electrochemistry#electrochemistry_212af058_3bbb_419f_a9c6_90ba9ebb3706',
 'PorousElectrodeCoating': 'http://emmo.info/electrochemistry#electrochemistry_403c300e_09b9_400b_943b_04e82a3cfb56',
 'MassFraction': 'http://emmo.info/emmo#EMMO_7c055d65_2929_40e1_af4f_4bf10995ad50',
 'Densit

In [3]:
list(uri_label_dict.keys())

['http://emmo.info/emmo#EMMO_168045f4_1ef4_4596_8115_1c68e150e9a2',
 'http://emmo.info/emmo#EMMO_c611d72b_0921_4c93_ab42_43b30084283e',
 'http://emmo.info/emmo#EMMO_c5ac5c53_14de_47ce_b424_2576d042d241',
 'http://emmo.info/emmo/domain/oie/manufacturing#Austenitising',
 'http://emmo.info/emmo#EMMO_0ef043ce_fcaa_43b9_b8b9_c1f8184d29c3',
 'http://emmo.info/emmo#EMMO_8bb34845_8b91_4127_970a_859b4541df73',
 'http://emmo.info/emmo#EMMO_ab3e812f_4d0f_4290_83fb_b2f5963f3772',
 'http://emmo.info/emmo/domain/oie/models#AtomisticPhononBasedModels-BoltzmannTransportEquation',
 'http://emmo.info/emmo/domain/periodic-table#EMMO_96522455-d7d7-5b1e-b10a-ad87e41a5adc',
 'https://big-map.github.io/BattINFO/ontology/BattINFO#EMMO_b5910d2f_3114_4663_8d46_e418ff859251',
 'https://big-map.github.io/BattINFO/ontology/electrochemistry#EMMO_dd15b4b0-11e7-4900-b379-9702a8caa6bb',
 'http://emmo.info/emmo#EMMO_209ba1b3_149f_4ff0_b672_941610eafd72',
 'http://emmo.info/emmo/domain/periodic-table#EMMO_41dba862-503f-

In [4]:
utils.iri_to_preflabel(rdf_graph=ontologies, iri="http://emmo.info/emmo#EMMO_4950a41c_ae32_4ce9_9db9_0076bbcac18f")

c:\Users\eibarc\OneDrive - SINTEF\Documents\Project BIGMAP\Repos\temp\gui\utils.py:28: UserWarning: The supplied IRI http://emmo.info/emmo#EMMO_4950a41c_ae32_4ce9_9db9_0076bbcac18f has 0 prefLabels: []
  warnings.warn(f"""The supplied IRI {iri} has {len(list(prefLabel))} prefLabels: {list(prefLabel)}""")


''

In [3]:
utils.preflabel_to_iri(rdf_graph=g, prefLabel="Anode", lang="en")

'https://big-map.github.io/BattINFO/ontology/electrochemistry#EMMO_b6319c74_d2ce_48c0_a75a_63156776b302'

In [3]:
utils.all_types(iri_label_dict=uri_label_dict, metadata_graph = g_metadata)

{'BatteryCell': 'http://emmo.info/battery#battery_68ed592a_7924_45d0_a108_94d6275d57f0',
 'Mass': 'http://emmo.info/emmo#EMMO_ed4af7ae_63a2_497e_bb88_2309619ea405',
 'Real': 'http://emmo.info/emmo#EMMO_18d180e4_5e3e_42f7_820c_e08951223486',
 'Width': 'http://emmo.info/emmo#EMMO_e4de48b1_dabb_4490_ac2b_040f926c64f0',
 'Height': 'http://emmo.info/emmo#EMMO_08bcf1d6_e719_46c8_bb21_24bc9bf34dba',
 'Thickness': 'http://emmo.info/emmo#EMMO_43003c86_9d15_433b_9789_ee2940920656',
 'PositiveElectrode': 'http://emmo.info/electrochemistry#electrochemistry_aff732a9_238a_4734_977c_b2ba202af126',
 'NumberOfEntities': 'http://emmo.info/emmo#EMMO_41efdf5d_0c9c_4ea0_bb65_f8236e663be5',
 'CurrentCollector': 'http://emmo.info/electrochemistry#electrochemistry_212af058_3bbb_419f_a9c6_90ba9ebb3706',
 'PorousElectrodeCoating': 'http://emmo.info/electrochemistry#electrochemistry_403c300e_09b9_400b_943b_04e82a3cfb56',
 'MassFraction': 'http://emmo.info/emmo#EMMO_7c055d65_2929_40e1_af4f_4bf10995ad50',
 'Densit

In [7]:
type_iri_labels_dict = {iri:uri_label_dict[iri] for iri in utils.all_types(rdf_graph=g_metadata) if iri in uri_label_dict}
type_iri_labels_dict

{'http://emmo.info/electrochemistry#electrochemistry_403c300e_09b9_400b_943b_04e82a3cfb56': 'PorousElectrodeCoating',
 'http://emmo.info/emmo#EMMO_d5be1faf_0c56_4f5a_9b78_581e6dee949f': 'AmountConcentration',
 'http://emmo.info/emmo#EMMO_43003c86_9d15_433b_9789_ee2940920656': 'Thickness',
 'http://emmo.info/emmo#EMMO_06448f64_8db6_4304_8b2c_e785dba82044': 'Density',
 'http://emmo.info/emmo#EMMO_41efdf5d_0c9c_4ea0_bb65_f8236e663be5': 'NumberOfEntities',
 'http://emmo.info/battery#battery_68ed592a_7924_45d0_a108_94d6275d57f0': 'BatteryCell',
 'http://emmo.info/electrochemistry#electrochemistry_1e3dc60d_dd6b_47d6_8161_70004fc5ee30': 'SpecificCapacity',
 'http://emmo.info/emmo#EMMO_e4de48b1_dabb_4490_ac2b_040f926c64f0': 'Width',
 'http://emmo.info/emmo#EMMO_18d180e4_5e3e_42f7_820c_e08951223486': 'Real',
 'http://emmo.info/electrochemistry#electrochemistry_212af058_3bbb_419f_a9c6_90ba9ebb3706': 'CurrentCollector',
 'http://emmo.info/electrochemistry#electrochemistry_331e6cca_f260_4bf8_af55_

In [8]:
for subj, pred in g.subject_predicates(object=URIRef("https://big-map.github.io/BattINFO/ontology/electrochemistry#EMMO_b6319c74_d2ce_48c0_a75a_63156776b302")):
    print(subj, pred)

https://big-map.github.io/BattINFO/ontology/electrochemistry#EMMO_35c650ab_3b23_4938_b312_1b0dede2e6d5 http://www.w3.org/2002/07/owl#disjointWith
nb65a59ebafcc4d499133876239b442ccb1086 http://www.w3.org/2002/07/owl#annotatedSource
